# Objectives 
 - Map how current News websites are to trending twitter topics
 - Websites used
     - BBC News
     - Telegraph
     - The Guardian
     - The Independent
     - Bloomberg


In [ ]:
from textblob import TextBlob
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from operator import itemgetter
from wordcloud import WordCloud
from pathlib import Path
# handling images
# https://stackoverflow.com/questions/35286540/display-an-image-with-python
import matplotlib.image as mpimg
import preprocessor as p

# Assignment 3

Assignment_03 : project Guternburg  pick 2 books and do 2 analyses and create a barplot of the top 30 popular words
whether they use the same words and create a barplot of the top 30 used words

# Beyond Good and Evil

In [ ]:
BGAE_blob=TextBlob(Path('BeyondGoodAndEvil.txt').read_text())

In [ ]:
BGAE_items=BGAE_blob.word_counts.items()

In [ ]:
stop_words=stopwords.words('english')

In [ ]:
#elimiate stop words
BGAE_items =[item for item in BGAE_items if item[0] not in stop_words]

In [ ]:
BGAE_sorted_items=sorted(BGAE_items, key=itemgetter(1), reverse=True)

In [ ]:
BGAE_top20=BGAE_sorted_items[:20]

In [ ]:
BGAE_df=pd.DataFrame(BGAE_top20, columns=['words','count'])

In [ ]:
axes=BGAE_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
#conver df to tuple
BGAE_tuples = [tuple(x) for x in BGAE_df.values]

In [ ]:
BGAE_wordcloud=WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(BGAE_tuples))

In [ ]:
BGAE_wordcloud_1 = BGAE_wordcloud.to_file('BGAE.png')

In [ ]:
plt.imshow(BGAE_wordcloud_1)

# Dracula

In [ ]:
DRACULA_blob=TextBlob(Path('Dracula.txt').read_text())

In [ ]:
DRACULA_items=DRACULA_blob.word_counts.items()

In [ ]:
DRACULA_stop_words=stopwords.words('english')

In [ ]:
#elimiate stop words
DRACULA_items =[item for item in DRACULA_items if item[0] not in DRACULA_stop_words]

In [ ]:
DRACULA_sorted_items=sorted(DRACULA_items, key=itemgetter(1), reverse=True)

In [ ]:
DRACULA_top20=DRACULA_sorted_items[:20]

In [ ]:
DRACULA_df=pd.DataFrame(DRACULA_top20, columns=['words','count'])

In [ ]:
axes=DRACULA_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
#conver df to tuple
DRACULA_tuples = [tuple(x) for x in DRACULA_df.values]

In [ ]:
DRACULA_wordcloud=WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(DRACULA_tuples))

In [ ]:
DRACULA_wordcloud_1 = DRACULA_wordcloud.to_file('BGAE.png')

In [ ]:
plt.imshow(DRACULA_wordcloud_1)

# Assignment 4

 Objectives
   + stream tweets for at least 5 sessions
   + create wordcloud for each session
   + create a bar chart for each session
   + Get from news media - maps with what is current 
   + which chart is almost the same with the twitter 

In [ ]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

In [ ]:
import keys

In [ ]:
auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token,keys.access_token_secret)

# Rate limiting

In [ ]:
api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

# Session 1

In [ ]:
import tweepy

#https://stackoverflow.com/questions/30721567/limit-tweepy-stream-to-a-specific-number
#https://www.storybench.org/how-to-collect-tweets-from-the-twitter-streaming-api-using-python/

# authorization tokens
#consumer_key = "[insert your key here]"
#consumer_secret = "[insert your secret here]"
#access_key = "[insert your key here]"
#access_secret = "[insert your secret here]"

# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    
    def __init__(self):
        super().__init__()
        self.tweet_count=0
        self.TWEET_LIMIT = 19 
    def on_status(self, status):
        #self.tweet_count=0
        #self.TWEET_LIMIT = limit
        print(status.id_str)
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")

        with open("tweet1.txt", "a", encoding='utf-8') as f:
            f.write("%s\n" % (text))
        self.tweet_count += 1
        #return self.tweet_count <= self.TWEET_LIMIT
        if self.tweet_count <= self.TWEET_LIMIT:
            return True
        else:
            # close file? nah
            # https://stackoverflow.com/questions/33498975/unable-to-stop-streaming-in-tweepy-after-one-minute
            # returining false to close stream
            return False
    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

if __name__ == "__main__":
    # complete authorization and initialize API endpoint
    auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
    auth.set_access_token(keys.access_token,keys.access_token_secret)
    api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
  #  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  #  auth.set_access_token(access_key, access_secret)
   # api = tweepy.API(auth)

    # initialize stream
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
    with open("tweet1.txt", "w", encoding='utf-8') as f:
        f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
    tags = ["United Kingdom"]
    stream.filter(track=tags)

In [ ]:
tweet1_text= Path('tweet1.txt').read_text(errors='ignore')

Preprocessor options

Option Name	Option Short Code

URL   	p.OPT.URL

Mention	p.OPT.MENTION

Hashtag	p.OPT.HASHTAG

Reserved Words	p.OPT.RESERVED

Emoji	p.OPT.EMOJI

Smiley	p.OPT.SMILEY

Number	p.OPT.NUMBER

In [ ]:
#all Preprocessor options enabled
p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG,p.OPT.MENTION,p.OPT.NUMBER)

In [ ]:
clean_tweet1_text=p.clean(tweet1_text)

In [ ]:
tweet1_blob=TextBlob(clean_tweet1_text)

In [ ]:
tweet1_items=tweet1_blob.word_counts.items()

In [ ]:
tweet1_stop_words=stopwords.words('english')

Adding words to the stopword list that were not blocked by the preprocessor

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
tweet1_stopwords_list = ['RT','ðÿ‡¬ðÿ‡§ðÿ‡ºðÿ‡¸	','ukâ€	','“','rt','date,user,is_retweet,is_quote,text,quoted_text','@','United','Kingdom']
tweet1_stop_words.extend(tweet1_stopwords_list)

In [ ]:
#elimiate stop words
tweet1_items =[item for item in tweet1_items if item[0] not in tweet1_stop_words]

In [ ]:
tweet1_sorted_items=sorted(tweet1_items, key=itemgetter(1), reverse=True)

In [ ]:
tweet1_top20=tweet1_sorted_items[:20]

In [ ]:
tweet1_df=pd.DataFrame(tweet1_top20, columns=['words','count'])

In [ ]:
tweet1_df

bar chart

In [ ]:
axes=tweet1_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
tweet1_tuples = [tuple(x) for x in tweet1_df.values]

In [ ]:
tweet1_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(tweet1_tuples))

In [ ]:
tweet1_wordcloud_1 = tweet1_wordcloud1.to_file('tweet1.png')

In [ ]:
plt.imshow(tweet1_wordcloud1)

# Compare BBC News Session 1

In [ ]:
import requests

In [ ]:
response=requests.get('https://www.bbc.com/news/uk')

In [ ]:
BBC_URL = 'https://www.bbc.com/news/uk'
BBC_page = requests.get(BBC_URL)
BBC_content = BBC_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(BBC_content, 'html.parser')

In [ ]:
BBC_text = soup.get_text(strip=True)

In [ ]:
BBC_blob=TextBlob(BBC_text)

In [ ]:
BBC_items=BBC_blob.word_counts.items()

In [ ]:
BBC_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
BBC_stopwords_list = ['optionsshare','toolsfacebooktwittershareview','morenextarticle','linkread',
                      'links.posted','hours','minutes','bbc','share','postcopy','news','contentnew']
BBC_stopwords.extend(BBC_stopwords_list)

In [ ]:
#elimiate stop words
BBC_items =[item for item in BBC_items if item[0] not in BBC_stopwords]

In [ ]:
BBC_sorted_items=sorted(BBC_items, key=itemgetter(1), reverse=True)

In [ ]:
BBC_sorted_items[len(BBC_sorted_items)-1]

In [ ]:
len(BBC_sorted_items)

In [ ]:
# limiting the array to  20 items
BBC_top20=BBC_sorted_items[:20]

In [ ]:
BBC_df=pd.DataFrame(BBC_top20, columns=['words','count'])

In [ ]:
axes=BBC_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
BBC_tuples = [tuple(x) for x in BBC_df.values]

In [ ]:
BBC_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(BBC_tuples))

In [ ]:
BBC_wordcloud_1 = BBC_wordcloud1.to_file('BBC1.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(BBC_wordcloud1)

# Telegraph Comparison Session 1

In [ ]:
import requests

In [ ]:
Tele_URL = 'https://www.Telegraph.co.uk/'
Tele_page = requests.get(Tele_URL)
Tele_content = Tele_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(Tele_content, 'html.parser')

In [ ]:
Tele_text = soup.get_text(strip=True)

In [ ]:
Tele_blob=TextBlob(Tele_text)

In [ ]:
Tele_items=Tele_blob.word_counts.items()

In [ ]:
Tele_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
Tele_stopwords_list = ['’','–','‘','Telegraph']
Tele_stopwords.extend(Tele_stopwords_list)

In [ ]:
#elimiate stop words
Tele_items =[item for item in Tele_items if item[0] not in Tele_stopwords]

In [ ]:
Tele_sorted_items=sorted(Tele_items, key=itemgetter(1), reverse=True)

In [ ]:
Tele_sorted_items[len(Tele_sorted_items)-1]

In [ ]:
len(Tele_sorted_items)

In [ ]:
# limiting the array to  20 items
Tele_top20=Tele_sorted_items[:20]

In [ ]:
Tele_df=pd.DataFrame(Tele_top20, columns=['words','count'])

In [ ]:
axes=Tele_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
Tele_tuples = [tuple(x) for x in Tele_df.values]

In [ ]:
Tele_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(Tele_tuples))

In [ ]:
Tele_wordcloud_1 = Tele_wordcloud1.to_file('Tele1.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Tele_wordcloud1)


# Session 2

In [ ]:
import keys

In [ ]:
auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token,keys.access_token_secret)

# Rate limiting

In [ ]:
api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
import tweepy

#https://stackoverflow.com/questions/30721567/limit-tweepy-stream-to-a-specific-number
#https://www.storybench.org/how-to-collect-tweets-from-the-twitter-streaming-api-using-python/

# authorization tokens
#consumer_key = "[insert your key here]"
#consumer_secret = "[insert your secret here]"
#access_key = "[insert your key here]"
#access_secret = "[insert your secret here]"

# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    
    def __init__(self):
        super().__init__()
        self.tweet_count=0
        self.TWEET_LIMIT = 19 
    def on_status(self, status):
        #self.tweet_count=0
        #self.TWEET_LIMIT = limit
        print(status.id_str)
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")

        with open("tweet2.txt", "a", encoding='utf-8') as f:
            f.write("%s\n" % (text))
        self.tweet_count += 1
        #return self.tweet_count <= self.TWEET_LIMIT
        if self.tweet_count <= self.TWEET_LIMIT:
            return True
        else:
            # close file? nah
            # https://stackoverflow.com/questions/33498975/unable-to-stop-streaming-in-tweepy-after-one-minute
            # returining false to close stream
            return False
    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

if __name__ == "__main__":
    # complete authorization and initialize API endpoint
    auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
    auth.set_access_token(keys.access_token,keys.access_token_secret)
    api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
  #  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  #  auth.set_access_token(access_key, access_secret)
   # api = tweepy.API(auth)

    # initialize stream
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
    with open("tweet2.txt", "w", encoding='utf-8') as f:
        f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
    tags = ["United Kingdom"]
    stream.filter(track=tags)

In [ ]:
tweet2_text= Path('tweet2.txt').read_text(errors='ignore')

Preprocessor options

Option Name	Option Short Code

URL   	p.OPT.URL

Mention	p.OPT.MENTION

Hashtag	p.OPT.HASHTAG

Reserved Words	p.OPT.RESERVED

Emoji	p.OPT.EMOJI

Smiley	p.OPT.SMILEY

Number	p.OPT.NUMBER

In [ ]:
#all Preprocessor options enabled
p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG,p.OPT.MENTION,p.OPT.NUMBER)

In [ ]:
clean_tweet2_text=p.clean(tweet2_text)

In [ ]:
tweet2_blob=TextBlob(clean_tweet2_text)

In [ ]:
tweet2_items=tweet2_blob.word_counts.items()

In [ ]:
tweet2_stop_words=stopwords.words('english')

Adding words to the stopword list that were not blocked by the preprocessor

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
tweet2_stopwords_list = ['RT','ðÿ‡¬ðÿ‡§ðÿ‡ºðÿ‡¸	','ukâ€	','“','rt','date,user,is_retweet,is_quote,text,quoted_text','@','United','Kingdom']
tweet2_stop_words.extend(tweet2_stopwords_list)

In [ ]:
#elimiate stop words
tweet2_items =[item for item in tweet2_items if item[0] not in tweet2_stop_words]

In [ ]:
tweet2_sorted_items=sorted(tweet2_items, key=itemgetter(1), reverse=True)

In [ ]:
tweet2_top20=tweet2_sorted_items[:20]

In [ ]:
tweet2_df=pd.DataFrame(tweet2_top20, columns=['words','count'])

bar chart

In [ ]:
axes=tweet2_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
#convert dataframe to tuble for word cloud to work
tweet2_tuples = [tuple(x) for x in tweet2_df.values]

In [ ]:
tweet2_wordcloud = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(tweet2_tuples))

In [ ]:
tweet2_wordcloud_1 = tweet2_wordcloud.to_file('tweet2.png')

In [ ]:
plt.imshow(tweet2_wordcloud)

# Compare BBC News Session 2

In [ ]:
import requests

In [ ]:
response=requests.get('https://www.bbc.com/news/uk')

In [ ]:
BBC2_URL = 'https://www.bbc.com/news/uk'
BBC2_page = requests.get(BBC2_URL)
BBC2_content = BBC2_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(BBC2_content, 'html.parser')

In [ ]:
BBC2_text = soup.get_text(strip=True)

In [ ]:
BBC2_blob=TextBlob(BBC2_text)

In [ ]:
BBC2_items=BBC2_blob.word_counts.items()

In [ ]:
BBC2_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
BBC2_stopwords_list = ['optionsshare','toolsfacebooktwittershareview','morenextarticle','linkread',
                      'links.posted','hours','minutes','bbc','share','postcopy','news','contentnew']
BBC2_stopwords.extend(BBC2_stopwords_list)

In [ ]:
#elimiate stop words
BBC2_items =[item for item in BBC2_items if item[0] not in BBC2_stopwords]

In [ ]:
BBC2_sorted_items=sorted(BBC2_items, key=itemgetter(1), reverse=True)

In [ ]:
BBC2_sorted_items[len(BBC2_sorted_items)-1]

In [ ]:
len(BBC2_sorted_items)

In [ ]:
# limiting the array to  20 items
BBC2_top20=BBC2_sorted_items[:20]

In [ ]:
BBC2_df=pd.DataFrame(BBC2_top20, columns=['words','count'])

In [ ]:
axes=BBC2_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
BBC2_tuples = [tuple(x) for x in BBC2_df.values]

In [ ]:
BBC2_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(BBC2_tuples))

In [ ]:
BBC2_wordcloud_1 = BBC2_wordcloud1.to_file('BBC2.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(BBC2_wordcloud1)

# Telegraph Comparison Session 2

In [ ]:
import requests

In [ ]:
Tele2_URL = 'https://www.Telegraph.co.uk/'
Tele2_page = requests.get(Tele2_URL)
Tele2_content = Tele2_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(Tele2_content, 'html.parser')

In [ ]:
Tele2_text = soup.get_text(strip=True)

In [ ]:
Tele2_blob=TextBlob(Tele2_text)

In [ ]:
Tele2_items=Tele2_blob.word_counts.items()

In [ ]:
Tele2_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
Tele2_stopwords_list = ['’','–','‘','Telegraph']
Tele2_stopwords.extend(Tele2_stopwords_list)

In [ ]:
#elimiate stop words
Tele2_items =[item for item in Tele2_items if item[0] not in Tele2_stopwords]

In [ ]:
Tele2_sorted_items=sorted(Tele2_items, key=itemgetter(1), reverse=True)

In [ ]:
Tele2_sorted_items[len(Tele2_sorted_items)-1]

In [ ]:
len(Tele2_sorted_items)

In [ ]:
# limiting the array to  20 items
Tele2_top20=Tele2_sorted_items[:20]

In [ ]:
Tele2_df=pd.DataFrame(Tele2_top20, columns=['words','count'])

In [ ]:
axes=Tele2_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
Tele2_tuples = [tuple(x) for x in Tele2_df.values]

In [ ]:
Tele2_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(Tele2_tuples))

In [ ]:
Tele2_wordcloud_1 = Tele2_wordcloud1.to_file('Tele2.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Tele2_wordcloud1)


# Session 3

In [ ]:
import keys

In [ ]:
auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token,keys.access_token_secret)

# Rate limiting

In [ ]:
api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
import tweepy

#https://stackoverflow.com/questions/30721567/limit-tweepy-stream-to-a-specific-number
#https://www.storybench.org/how-to-collect-tweets-from-the-twitter-streaming-api-using-python/

# authorization tokens
#consumer_key = "[insert your key here]"
#consumer_secret = "[insert your secret here]"
#access_key = "[insert your key here]"
#access_secret = "[insert your secret here]"

# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    
    def __init__(self):
        super().__init__()
        self.tweet_count=0
        self.TWEET_LIMIT = 19 
    def on_status(self, status):
        #self.tweet_count=0
        #self.TWEET_LIMIT = limit
        print(status.id_str)
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")

        with open("tweet3.txt", "a", encoding='utf-8') as f:
            f.write("%s\n" % (text))
        self.tweet_count += 1
        #return self.tweet_count <= self.TWEET_LIMIT
        if self.tweet_count <= self.TWEET_LIMIT:
            return True
        else:
            # close file? nah
            # https://stackoverflow.com/questions/33498975/unable-to-stop-streaming-in-tweepy-after-one-minute
            # returining false to close stream
            return False
    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

if __name__ == "__main__":
    # complete authorization and initialize API endpoint
    auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
    auth.set_access_token(keys.access_token,keys.access_token_secret)
    api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
  #  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  #  auth.set_access_token(access_key, access_secret)
   # api = tweepy.API(auth)

    # initialize stream
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
    with open("tweet3.txt", "w", encoding='utf-8') as f:
        f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
    tags = ["United Kingdom"]
    stream.filter(track=tags)

In [ ]:
tweet3_text= Path('tweet3.txt').read_text(errors='ignore')

Preprocessor options

Option Name	Option Short Code

URL   	p.OPT.URL

Mention	p.OPT.MENTION

Hashtag	p.OPT.HASHTAG

Reserved Words	p.OPT.RESERVED

Emoji	p.OPT.EMOJI

Smiley	p.OPT.SMILEY

Number	p.OPT.NUMBER

In [ ]:
#all Preprocessor options enabled
p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG,p.OPT.MENTION,p.OPT.NUMBER)

In [ ]:
clean_tweet3_text=p.clean(tweet3_text)

In [ ]:
tweet3_blob=TextBlob(clean_tweet3_text)

In [ ]:
tweet3_items=tweet3_blob.word_counts.items()

In [ ]:
tweet3_stop_words=stopwords.words('english')

Adding words to the stopword list that were not blocked by the preprocessor

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
tweet3_stopwords_list = ['RT','ðÿ‡¬ðÿ‡§ðÿ‡ºðÿ‡¸	','ukâ€	','“','rt','date,user,is_retweet,is_quote,text,quoted_text','@','United','Kingdom']
tweet3_stop_words.extend(tweet3_stopwords_list)

In [ ]:
#elimiate stop words
tweet3_items =[item for item in tweet3_items if item[0] not in tweet3_stop_words]

In [ ]:
tweet3_sorted_items=sorted(tweet3_items, key=itemgetter(1), reverse=True)

In [ ]:
tweet3_top20=tweet3_sorted_items[:20]

In [ ]:
tweet3_df=pd.DataFrame(tweet3_top20, columns=['words','count'])

bar chart

In [ ]:
axes=tweet3_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
#convert dataframe to tuble for word cloud to work
tweet3_tuples = [tuple(x) for x in tweet3_df.values]

In [ ]:
tweet3_wordcloud = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(tweet3_tuples))

In [ ]:
tweet3_wordcloud_1 = tweet3_wordcloud.to_file('tweet3.png')

In [ ]:
plt.imshow(tweet3_wordcloud)

# Compare BBC News Session 3

In [ ]:
import requests

In [ ]:
response=requests.get('https://www.bbc.com/news/uk')

In [ ]:
BBC3_URL = 'https://www.bbc.com/news/uk'
BBC3_page = requests.get(BBC3_URL)
BBC3_content = BBC3_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(BBC3_content, 'html.parser')

In [ ]:
BBC3_text = soup.get_text(strip=True)

In [ ]:
BBC3_blob=TextBlob(BBC3_text)

In [ ]:
BBC3_items=BBC3_blob.word_counts.items()

In [ ]:
BBC3_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
BBC3_stopwords_list = ['optionsshare','toolsfacebooktwittershareview','morenextarticle','linkread',
                      'links.posted','hours','minutes','bbc','share','postcopy','news','contentnew']
BBC3_stopwords.extend(BBC3_stopwords_list)

In [ ]:
#elimiate stop words
BBC3_items =[item for item in BBC3_items if item[0] not in BBC3_stopwords]

In [ ]:
BBC3_sorted_items=sorted(BBC3_items, key=itemgetter(1), reverse=True)

In [ ]:
BBC3_sorted_items[len(BBC3_sorted_items)-1]

In [ ]:
len(BBC3_sorted_items)

In [ ]:
# limiting the array to  20 items
BBC3_top20=BBC3_sorted_items[:20]

In [ ]:
BBC3_df=pd.DataFrame(BBC3_top20, columns=['words','count'])

In [ ]:
axes=BBC3_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
BBC3_tuples = [tuple(x) for x in BBC3_df.values]

In [ ]:
BBC3_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(BBC3_tuples))

In [ ]:
BBC3_wordcloud_1 = BBC3_wordcloud1.to_file('BBC3.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(BBC3_wordcloud1)

# Telegraph Comparison Session 3

In [ ]:
import requests

In [ ]:
Tele3_URL = 'https://www.Telegraph.co.uk/'
Tele3_page = requests.get(Tele3_URL)
Tele3_content = Tele3_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(Tele3_content, 'html.parser')

In [ ]:
Tele3_text = soup.get_text(strip=True)

In [ ]:
Tele3_blob=TextBlob(Tele3_text)

In [ ]:
Tele3_items=Tele3_blob.word_counts.items()

In [ ]:
Tele3_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
Tele3_stopwords_list = ['’','–','‘','Tele3graph']
Tele3_stopwords.extend(Tele3_stopwords_list)

In [ ]:
#elimiate stop words
Tele3_items =[item for item in Tele3_items if item[0] not in Tele3_stopwords]

In [ ]:
Tele3_sorted_items=sorted(Tele3_items, key=itemgetter(1), reverse=True)

In [ ]:
Tele3_sorted_items[len(Tele3_sorted_items)-1]

In [ ]:
len(Tele3_sorted_items)

In [ ]:
# limiting the array to  20 items
Tele3_top20=Tele3_sorted_items[:20]

In [ ]:
Tele3_df=pd.DataFrame(Tele3_top20, columns=['words','count'])

In [ ]:
axes=Tele3_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
Tele3_tuples = [tuple(x) for x in Tele3_df.values]

In [ ]:
Tele3_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(Tele3_tuples))

In [ ]:
Tele3_wordcloud_1 = Tele3_wordcloud1.to_file('Tele3.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Tele3_wordcloud1)


# Session 4

In [ ]:
import keys

In [ ]:
auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token,keys.access_token_secret)

# Rate limiting

In [ ]:
api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
import tweepy

#https://stackoverflow.com/questions/30721567/limit-tweepy-stream-to-a-specific-number
#https://www.storybench.org/how-to-collect-tweets-from-the-twitter-streaming-api-using-python/

# authorization tokens
#consumer_key = "[insert your key here]"
#consumer_secret = "[insert your secret here]"
#access_key = "[insert your key here]"
#access_secret = "[insert your secret here]"

# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    
    def __init__(self):
        super().__init__()
        self.tweet_count=0
        self.TWEET_LIMIT = 19 
    def on_status(self, status):
        #self.tweet_count=0
        #self.TWEET_LIMIT = limit
        print(status.id_str)
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")

        with open("tweet4.txt", "a", encoding='utf-8') as f:
            f.write("%s\n" % (text))
        self.tweet_count += 1
        #return self.tweet_count <= self.TWEET_LIMIT
        if self.tweet_count <= self.TWEET_LIMIT:
            return True
        else:
            # close file? nah
            # https://stackoverflow.com/questions/33498975/unable-to-stop-streaming-in-tweepy-after-one-minute
            # returining false to close stream
            return False
    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

if __name__ == "__main__":
    # complete authorization and initialize API endpoint
    auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
    auth.set_access_token(keys.access_token,keys.access_token_secret)
    api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
  #  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  #  auth.set_access_token(access_key, access_secret)
   # api = tweepy.API(auth)

    # initialize stream
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
    with open("tweet4.txt", "w", encoding='utf-8') as f:
        f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
    tags = ["United Kingdom"]
    stream.filter(track=tags)

In [ ]:
tweet4_text= Path('tweet4.txt').read_text(errors='ignore')

Preprocessor options

Option Name	Option Short Code

URL   	p.OPT.URL

Mention	p.OPT.MENTION

Hashtag	p.OPT.HASHTAG

Reserved Words	p.OPT.RESERVED

Emoji	p.OPT.EMOJI

Smiley	p.OPT.SMILEY

Number	p.OPT.NUMBER

In [ ]:
#all Preprocessor options enabled
p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG,p.OPT.MENTION,p.OPT.NUMBER)

In [ ]:
clean_tweet4_text=p.clean(tweet4_text)

In [ ]:
tweet4_blob=TextBlob(clean_tweet4_text)

In [ ]:
tweet4_items=tweet4_blob.word_counts.items()

In [ ]:
tweet4_stop_words=stopwords.words('english')

Adding words to the stopword list that were not blocked by the preprocessor

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
tweet4_stopwords_list = ['RT','ðÿ‡¬ðÿ‡§ðÿ‡ºðÿ‡¸	','ukâ€	','“','rt','date,user,is_retweet,is_quote,text,quoted_text','@','United','Kingdom']
tweet4_stop_words.extend(tweet4_stopwords_list)

In [ ]:
#elimiate stop words
tweet4_items =[item for item in tweet4_items if item[0] not in tweet4_stop_words]

In [ ]:
tweet4_sorted_items=sorted(tweet4_items, key=itemgetter(1), reverse=True)

In [ ]:
tweet4_top20=tweet4_sorted_items[:20]

In [ ]:
tweet4_df=pd.DataFrame(tweet4_top20, columns=['words','count'])

bar chart

In [ ]:
axes=tweet4_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
#convert dataframe to tuble for word cloud to work
tweet4_tuples = [tuple(x) for x in tweet4_df.values]

In [ ]:
tweet4_wordcloud = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(tweet4_tuples))

In [ ]:
tweet4_wordcloud_1 = tweet4_wordcloud.to_file('tweet4.png')

In [ ]:
plt.imshow(tweet4_wordcloud)

# Compare BBC News Session 4

In [ ]:
import requests

In [ ]:
response=requests.get('https://www.bbc.com/news/uk')

In [ ]:
BBC4_URL = 'https://www.bbc.com/news/uk'
BBC4_page = requests.get(BBC4_URL)
BBC4_content = BBC4_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(BBC4_content, 'html.parser')

In [ ]:
BBC4_text = soup.get_text(strip=True)

In [ ]:
BBC4_blob=TextBlob(BBC4_text)

In [ ]:
BBC4_items=BBC4_blob.word_counts.items()

In [ ]:
BBC4_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
BBC4_stopwords_list = ['optionsshare','toolsfacebooktwittershareview','morenextarticle','linkread',
                      'links.posted','hours','minutes','bbc','share','postcopy','news','contentnew']
BBC4_stopwords.extend(BBC4_stopwords_list)

In [ ]:
#elimiate stop words
BBC4_items =[item for item in BBC4_items if item[0] not in BBC4_stopwords]

In [ ]:
BBC4_sorted_items=sorted(BBC4_items, key=itemgetter(1), reverse=True)

In [ ]:
BBC4_sorted_items[len(BBC4_sorted_items)-1]

In [ ]:
len(BBC4_sorted_items)

In [ ]:
# limiting the array to  20 items
BBC4_top20=BBC4_sorted_items[:20]

In [ ]:
BBC4_df=pd.DataFrame(BBC4_top20, columns=['words','count'])

In [ ]:
axes=BBC4_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
BBC4_tuples = [tuple(x) for x in BBC4_df.values]

In [ ]:
BBC4_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(BBC4_tuples))

In [ ]:
BBC4_wordcloud_1 = BBC4_wordcloud1.to_file('BBC4.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(BBC4_wordcloud1)

# Telegraph Comparison Session 4

In [ ]:
import requests

In [ ]:
Tele4_URL = 'https://www.Telegraph.co.uk/'
Tele4_page = requests.get(Tele4_URL)
Tele4_content = Tele4_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(Tele4_content, 'html.parser')

In [ ]:
Tele4_text = soup.get_text(strip=True)

In [ ]:
Tele4_blob=TextBlob(Tele4_text)

In [ ]:
Tele4_items=Tele4_blob.word_counts.items()

In [ ]:
Tele4_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
Tele4_stopwords_list = ['’','–','‘','Tele4graph']
Tele4_stopwords.extend(Tele4_stopwords_list)

In [ ]:
#elimiate stop words
Tele4_items =[item for item in Tele4_items if item[0] not in Tele4_stopwords]

In [ ]:
Tele4_sorted_items=sorted(Tele4_items, key=itemgetter(1), reverse=True)

In [ ]:
Tele4_sorted_items[len(Tele4_sorted_items)-1]

In [ ]:
len(Tele4_sorted_items)

In [ ]:
# limiting the array to  20 items
Tele4_top20=Tele4_sorted_items[:20]

In [ ]:
Tele4_df=pd.DataFrame(Tele4_top20, columns=['words','count'])

In [ ]:
axes=Tele4_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
Tele4_tuples = [tuple(x) for x in Tele4_df.values]

In [ ]:
Tele4_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(Tele4_tuples))

In [ ]:
Tele4_wordcloud_1 = Tele4_wordcloud1.to_file('Tele4.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Tele4_wordcloud1)


# Session 5

In [ ]:
import keys

In [ ]:
auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token,keys.access_token_secret)

# Rate limiting

In [ ]:
api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
import tweepy

#https://stackoverflow.com/questions/30721567/limit-tweepy-stream-to-a-specific-number
#https://www.storybench.org/how-to-collect-tweets-from-the-twitter-streaming-api-using-python/

# authorization tokens
#consumer_key = "[insert your key here]"
#consumer_secret = "[insert your secret here]"
#access_key = "[insert your key here]"
#access_secret = "[insert your secret here]"

# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    
    def __init__(self):
        super().__init__()
        self.tweet_count=0
        self.TWEET_LIMIT = 19 
    def on_status(self, status):
        #self.tweet_count=0
        #self.TWEET_LIMIT = limit
        print(status.id_str)
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")

        with open("tweet5.txt", "a", encoding='utf-8') as f:
            f.write("%s\n" % (text))
        self.tweet_count += 1
        #return self.tweet_count <= self.TWEET_LIMIT
        if self.tweet_count <= self.TWEET_LIMIT:
            return True
        else:
            # close file? nah
            # https://stackoverflow.com/questions/33498975/unable-to-stop-streaming-in-tweepy-after-one-minute
            # returining false to close stream
            return False
    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

if __name__ == "__main__":
    # complete authorization and initialize API endpoint
    auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
    auth.set_access_token(keys.access_token,keys.access_token_secret)
    api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
  #  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  #  auth.set_access_token(access_key, access_secret)
   # api = tweepy.API(auth)

    # initialize stream
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
    with open("tweet5.txt", "w", encoding='utf-8') as f:
        f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
    tags = ["United Kingdom"]
    stream.filter(track=tags)

In [ ]:
tweet5_text= Path('tweet5.txt').read_text(errors='ignore')

Preprocessor options

Option Name	Option Short Code

URL   	p.OPT.URL

Mention	p.OPT.MENTION

Hashtag	p.OPT.HASHTAG

Reserved Words	p.OPT.RESERVED

Emoji	p.OPT.EMOJI

Smiley	p.OPT.SMILEY

Number	p.OPT.NUMBER

In [ ]:
#all Preprocessor options enabled
p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG,p.OPT.MENTION,p.OPT.NUMBER)

In [ ]:
clean_tweet5_text=p.clean(tweet5_text)

In [ ]:
tweet5_blob=TextBlob(clean_tweet5_text)

In [ ]:
tweet5_items=tweet5_blob.word_counts.items()

In [ ]:
tweet5_stop_words=stopwords.words('english')

Adding words to the stopword list that were not blocked by the preprocessor

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
tweet5_stopwords_list = ['RT','ðÿ‡¬ðÿ‡§ðÿ‡ºðÿ‡¸	','ukâ€	','“','rt','date,user,is_retweet,is_quote,text,quoted_text','@','United','Kingdom']
tweet5_stop_words.extend(tweet5_stopwords_list)

In [ ]:
#elimiate stop words
tweet5_items =[item for item in tweet5_items if item[0] not in tweet5_stop_words]

In [ ]:
tweet5_sorted_items=sorted(tweet5_items, key=itemgetter(1), reverse=True)

In [ ]:
tweet5_top20=tweet5_sorted_items[:20]

In [ ]:
tweet5_df=pd.DataFrame(tweet5_top20, columns=['words','count'])

bar chart

In [ ]:
axes=tweet5_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
#convert dataframe to tuble for word cloud to work
tweet5_tuples = [tuple(x) for x in tweet5_df.values]

In [ ]:
tweet5_wordcloud = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(tweet5_tuples))

In [ ]:
tweet5_wordcloud_1 = tweet5_wordcloud.to_file('tweet5.png')

In [ ]:
plt.imshow(tweet5_wordcloud)

# Compare BBC News Session 5

In [ ]:
import requests

In [ ]:
response=requests.get('https://www.bbc.com/news/uk')

In [ ]:
BBC5_URL = 'https://www.bbc.com/news/uk'
BBC5_page = requests.get(BBC5_URL)
BBC5_content = BBC5_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(BBC5_content, 'html.parser')

In [ ]:
BBC5_text = soup.get_text(strip=True)

In [ ]:
BBC5_blob=TextBlob(BBC5_text)

In [ ]:
BBC5_items=BBC5_blob.word_counts.items()

In [ ]:
BBC5_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
BBC5_stopwords_list = ['optionsshare','toolsfacebooktwittershareview','morenextarticle','linkread',
                      'links.posted','hours','minutes','bbc','share','postcopy','news','contentnew']
BBC5_stopwords.extend(BBC5_stopwords_list)

In [ ]:
#elimiate stop words
BBC5_items =[item for item in BBC5_items if item[0] not in BBC5_stopwords]

In [ ]:
BBC5_sorted_items=sorted(BBC5_items, key=itemgetter(1), reverse=True)

In [ ]:
BBC5_sorted_items[len(BBC5_sorted_items)-1]

In [ ]:
len(BBC5_sorted_items)

In [ ]:
# limiting the array to  20 items
BBC5_top20=BBC5_sorted_items[:20]

In [ ]:
BBC5_df=pd.DataFrame(BBC5_top20, columns=['words','count'])

In [ ]:
axes=BBC5_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
BBC5_tuples = [tuple(x) for x in BBC5_df.values]

In [ ]:
BBC5_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(BBC5_tuples))

In [ ]:
BBC5_wordcloud_1 = BBC5_wordcloud1.to_file('BBC5.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(BBC5_wordcloud1)

# Telegraph Comparison  Session 5

In [ ]:
import requests

In [ ]:
Tele5_URL = 'https://www.Telegraph.co.uk/'
Tele5_page = requests.get(Tele5_URL)
Tele5_content = Tele5_page.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(Tele5_content, 'html.parser')

In [ ]:
Tele5_text = soup.get_text(strip=True)

In [ ]:
Tele5_blob=TextBlob(Tele5_text)

In [ ]:
Tele5_items=Tele5_blob.word_counts.items()

In [ ]:
Tele5_stopwords = stopwords.words('english')

# Adding words to the stopword list that were not included

In [ ]:
# https://stackabuse.com/removing-stop-words-from-strings-in-python/
Tele5_stopwords_list = ['’','–','‘','Tele5graph']
Tele5_stopwords.extend(Tele5_stopwords_list)

In [ ]:
#elimiate stop words
Tele5_items =[item for item in Tele5_items if item[0] not in Tele5_stopwords]

In [ ]:
Tele5_sorted_items=sorted(Tele5_items, key=itemgetter(1), reverse=True)

In [ ]:
# limiting the array to  20 items
Tele5_top20=Tele5_sorted_items[:20]

In [ ]:
Tele5_df=pd.DataFrame(Tele5_top20, columns=['words','count'])

In [ ]:
axes=Tele5_df.plot.bar(x='words',y='count', legend=False)

In [ ]:
Tele5_tuples = [tuple(x) for x in Tele5_df.values]

In [ ]:
Tele5_wordcloud1 = WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(Tele_tuples))

In [ ]:
Tele5_wordcloud_1 = Tele5_wordcloud1.to_file('Tele5.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Tele5_wordcloud1)


# Analysis

The word cloud supports that twitter is more current than news media websites as the words do not occur at the same frequency, However this comparison may be better suited with the use of the trends API which can be narrowed down to a particular region. The streamlistener api would be best suited for global events.